Para esta posible solución usamos monitores y semaforos como en la exposición de clase.

In [1]:
n_izq = int(input('Número de carros por la izquieda: '))
n_der = int(input('Número de carros por la derecha: '))

Número de carros por la izquieda: 3
Número de carros por la derecha: 3


In [2]:
import time #Importamos la libreria para el tiempo
import threading #Importamos la libreria para usar hilos
import sys #Importamos la libreria para manejar entorno

class Semaphore():

    'Crea semaforo para indicar cuando cada carro pueda pasar o no por el puente.'

    def __init__(self, initial):
        self.lock = threading.Condition() 
        self.value = initial

    'Para indicar que entra'

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() 

    'Para indicar que salen'

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() 
            self.value -= 1
            
class Puente():

    'Se crea la clase para el puente'

    def __init__(self):    
        self.lock = threading.Condition()
        self.taken = False

    'Metodo para indicar cuando un carro entra al puente'

    def take(self, sentido):         
        with self.lock:
            while self.taken == True:
                sys.stdout.write("___________________Carro [%s] esperando por entrar___________________\n" % (sentido))
                self.lock.wait()
            self.taken = True
            sys.stdout.write("___________________Carro [%s] pasando por el puente-----\n"%(sentido))
            sys.stdout.write("___________________Carro [%s] pasó por el puente___________________\n"%(sentido))
            self.lock.notifyAll()

    'Metodo para inventar cuando sale del puente'

    def drop(self, sentido):         
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.taken = False
            self.lock.notifyAll()
            
class Automovil_izq(threading.Thread):

    'Crea carro que entra por la izquierda con semaforo indicando cuando pueda entrar o no'

    def __init__(self, left, semaforo):
        threading.Thread.__init__(self)
        self.left = left
        self.semaforo = semaforo
    
    #Indica cuando entra y sale
    def run(self):
        while True:
            sys.stdout.write("Carro sentido izquierdo quiere pasar\n" )
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)
            self.left.take('izquierdo')        
            time.sleep(1)  #           
            self.left.drop('izquierdo')  
            time.sleep(1)
            self.semaforo.up()             
            time.sleep(1)
            
class Automovil_der(threading.Thread):

    'Crea carro que entra por la derecha con semaforo indicando cuando pueda entrar o no'

    def __init__(self, right, semaforo):
        threading.Thread.__init__(self)
        self.right = right
        self.semaforo = semaforo
   
    #Indica cuando entra y sale
    def run(self):
        for i in range(1):
            sys.stdout.write("Carro sentido derecho quiere pasar\n")
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)
            self.right.take('derecho')        
            time.sleep(1)  
            self.right.drop('derecho')  
            time.sleep(1)
            self.semaforo.up()             
            time.sleep(1)

def transito():

    'Simula comportamiento de puente'

    #Crea 4 carros por la izquierda y derecha 
    #n_izq = 4
    #n_der = 4

    print(f'Simulamos {n_izq} por la izquierda y {n_der} por la derecha.')

    #Creamos el comportamiento del puente para cada uno de los carros
    c = [Puente() for i in range(n_izq + n_der)]
    
    #Crea el semoforo
    semaforo = Semaphore(2)
    sys.stdout.write("\n")

    #Hace parejas con los autos y sus respectivos semaforos
    p_izq = [Automovil_izq(c[i], semaforo) for i in range(n_izq)]
    p_der = [Automovil_der(c[i], semaforo) for i in range(n_der)]

    #Mete carros al puente
    for i in range(n_izq):
        p_izq[i].start()
    for i in range(n_der):
        p_der[i].start()

    #Espera a que cada carro salga    
    for i in range(n_izq):
        p_der[i].join()
    for i in range(n_der):
        p_izq[i].join()
    
transito()

Simulamos 3 por la izquierda y 3 por la derecha.

Carro sentido izquierdo quiere pasar
Carro sentido izquierdo quiere pasar
Carro sentido izquierdo quiere pasar
Carro sentido derecho quiere pasar
Carro sentido derecho quiere pasar
Carro sentido derecho quiere pasar
___________________Carro [izquierdo] pasando por el puente-----
___________________Carro [izquierdo] pasó por el puente___________________
___________________Carro [izquierdo] pasando por el puente-----
___________________Carro [izquierdo] pasó por el puente___________________
Carro sentido izquierdo quiere pasar
___________________Carro [derecho] pasando por el puente-----
___________________Carro [derecho] pasó por el puente___________________
___________________Carro [izquierdo] pasando por el puente-----
___________________Carro [izquierdo] pasó por el puente___________________
Carro sentido izquierdo quiere pasar
___________________Carro [derecho] pasando por el puente-----
___________________Carro [derecho] pasó por el

KeyboardInterrupt: ignored